<a href="https://colab.research.google.com/github/crissstar/projectmanager/blob/main/2_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_%EC%98%88%EC%A0%9C_%EB%B9%84%EC%A0%95%ED%98%95%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B9%B4%EC%B9%B4%EC%98%A4%ED%86%A1_%EB%8C%80%ED%99%94%EB%B0%A9_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [비정형 데이터 -> 정형 데이터로 만들기] : 카카오톡 데이터 분석

- [참고] https://wikidocs.net/141389

## 0. 데이터 준비하기(모바일에서)

1. 분석할 특정 채팅방 선택하기
2. 화면 우측 상단 더보기(줄세개) 선택하기
3. 화면 우측 하단 채팅방 설정(톱니바퀴) 선택하기
4. 대화 내용 내보내기 선택 (텍스트 메시지만 보내기)
5. 메일 등을 이용하여 다운로드 받아 data 폴더로 이동시키기

## 1. 원본데이터 살펴보기

In [3]:
import pandas as pd

# 자신의 카카오톡 데이터 파일명으로 변경한다.
file = './sample_data/KakaoTalkChats.txt'
df = pd.read_table(file)
df

,"TMD-최수경-샘, 송혜경 3 님과 카카오톡 대화"
0,저장한 날짜 : 2023년 3월 23일 오후 3:52
1,2022년 10월 24일 오전 10:48
2,"2022년 10월 24일 오전 10:48, 조윤실님이 TMD-최수경-샘님과 송혜경님..."
3,"2022년 10월 24일 오전 10:48, 조윤실 : 혜경샘~~오늘 수경샘과 맛난 ..."
4,"2022년 10월 24일 오전 10:48, 조윤실 : 청계산 청룡공영주차장"
...,...
296,"2023년 3월 22일 오전 11:34, TMD-최수경-샘 : 그 친구는 만들고 싶..."
297,"2023년 3월 22일 오전 11:35, TMD-최수경-샘 : 점심 맛나게들 챙겨드세욤"
298,"2023년 3월 22일 오전 11:36, 송혜경 : 이모티콘"
299,"2023년 3월 22일 오전 11:39, 조윤실 : 난 지금부터 일 시작!!!"


## 2. 데이터 전처리(카카오톡 데이터 파싱하기)
- 참고:  https://regexr.com/

[예제 문장] 2023년 3월 22일 오전 11:36, 송혜경 : 이모티콘 (Android)

1. 카카오톡 데이터 쪼개기
    - 날짜와 시간: (iOS: . / Android: 년,월,일
    - 보낸 사람과 텍스트
2. 카카오톡 데이터 파싱하기
    - 정규표현식 매칭 & 메시지 파싱
3. 데이터 프레임으로 만들기

In [4]:
import re
import pandas as pd
import os


def katalk_msg_parse(file_path):
    my_katalk_data = list()

    # 정규 패턴 정의
    katalk_msg_pattern = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2},.*:"
    date_info = "[0-9]{4}년 [0-9]{1,2}월 [0-9]{1,2}일 \S요일" #0-9번까지 숫자중 4자리, 0-9번 중 1자리 또는 2자리 중에서 월, 일을 뽑음
    in_out_info = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2}:.*"

    # 파일 데이터 읽기
    cnt = 0
    f = open(file_path, encoding='utf-8')
    for line in f :
        cnt += 1
        if re.match(date_info, line) or re.match(in_out_info, line):
            continue
        elif line == '\n': # 엔터
            continue
        elif re.match(katalk_msg_pattern, line):  # 카카오톡 메시지 패턴
            line = line.split(",")
            date_time = line[0]                           # 년월일 시간
            user_text = line[1].split(" : ", maxsplit=1)
            user_name = user_text[0].strip()              # 메시지 보낸 사람
            text = user_text[1].strip()                   # 메시지
            # DataFrame으로 나타내기 위해 my_katalk_data 리스트에 딕셔너리 타입으로 추가하기
            my_katalk_data.append({'date_time': date_time,
                                   'user_name': user_name,
                                   'text': text
                                   })

        else:
            if len(my_katalk_data) > 0:
                my_katalk_data[-1]['text'] += "\n"+line.strip()
    print(f'파일 read count: {cnt}')

    # DataFrame 데이터로 만들기
    my_katalk_df = pd.DataFrame(my_katalk_data)
    print(f'DataFrame count: {len(my_katalk_df)}')
    return my_katalk_df


df = katalk_msg_parse(file)

df.head()

파일 read count: 328
DataFrame count: 249


,date_time,user_name,text
0,2022년 10월 24일 오전 10:48,조윤실,혜경샘~~오늘 수경샘과 맛난 점심하러 가겠습니다~
1,2022년 10월 24일 오전 10:48,조윤실,청계산 청룡공영주차장\n서울 서초구 신원동 224-3
2,2022년 10월 24일 오전 10:49,조윤실,수경샘~ 작은 주차장이지만 여기에 주차하시면됩니다.
3,2022년 10월 24일 오전 10:50,조윤실,저는 12시 10분 땡퇴하고 가겠습니다. 25~30분 정도 걸릴거에요.
4,2022년 10월 24일 오전 11:09,TMD-최수경-샘,넹!! 곧 뵈어요


## 3. 데이터 전처리(컬럼 분할)

### 컬럼 추가하기

In [5]:
df['datetime'] = df['date_time']

### 컬럼 순서 변경하기

In [6]:
df.columns
df = df[ ['date_time', 'datetime', 'user_name', 'text'] ]

### 날짜, 시간 세분화하기

In [7]:
# 오전/오후 --> AM/PM 으로 변경
df['datetime'] = df['datetime'].str.replace('오전', 'AM')
df['datetime'] = df['datetime'].str.replace('오후', 'PM')
df.head(3)

,date_time,datetime,user_name,text
0,2022년 10월 24일 오전 10:48,2022년 10월 24일 AM 10:48,조윤실,혜경샘~~오늘 수경샘과 맛난 점심하러 가겠습니다~
1,2022년 10월 24일 오전 10:48,2022년 10월 24일 AM 10:48,조윤실,청계산 청룡공영주차장\n서울 서초구 신원동 224-3
2,2022년 10월 24일 오전 10:49,2022년 10월 24일 AM 10:49,조윤실,수경샘~ 작은 주차장이지만 여기에 주차하시면됩니다.


In [ ]:
# pandas 표준 date_time 포맷으로 변경
# df['datetime'] = pd.to_datetime(df['datetime'], format='%Y. %m. %d. %p %I:%M')
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y년 %m월 %d일 %p %H:%M')
df.head(3)

In [ ]:
# text	date	year	month	day	weekday	hour 컬럼 추가
df['date'] = df['datetime'].dt.date
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['weekday'] = df['datetime'].dt.day_name()
df['hour'] = df['datetime'].dt.hour
df.head(3)

### 카카오톡 메시지 길이

In [8]:
# 메시지 길이, 메시지 단어 개수  컬럼 추가
df['msg_len'] = df['text'].str.len()  # 메시지 길이
df['msg_word_count'] = df['text'].str.split().str.len() #메시지 단어 개수
df.head()

,date_time,datetime,user_name,text,msg_len,msg_word_count
0,2022년 10월 24일 오전 10:48,2022년 10월 24일 AM 10:48,조윤실,혜경샘~~오늘 수경샘과 맛난 점심하러 가겠습니다~,27,5
1,2022년 10월 24일 오전 10:48,2022년 10월 24일 AM 10:48,조윤실,청계산 청룡공영주차장\n서울 서초구 신원동 224-3,28,6
2,2022년 10월 24일 오전 10:49,2022년 10월 24일 AM 10:49,조윤실,수경샘~ 작은 주차장이지만 여기에 주차하시면됩니다.,30,5
3,2022년 10월 24일 오전 10:50,2022년 10월 24일 AM 10:50,조윤실,저는 12시 10분 땡퇴하고 가겠습니다. 25~30분 정도 걸릴거에요.,39,8
4,2022년 10월 24일 오전 11:09,2022년 10월 24일 AM 11:09,TMD-최수경-샘,넹!! 곧 뵈어요,9,3


### 사진, 동영상 정보 추출

In [14]:
audio_visual_text = '^동영상$|^사진$|^사진 [0-9]{1,2}장$'
mask = df['text'].str.contains(audio_visual_text)
df.loc[mask, 'audio_visual'] = 1
df.loc[~mask, 'audio_visual'] = 0
df

,date_time,datetime,user_name,text,msg_len,msg_word_count,facetalk,voicetalk,facetalk,voicetalk,audio_visual
0,2022년 10월 24일 오전 10:48,2022년 10월 24일 AM 10:48,조윤실,혜경샘~~오늘 수경샘과 맛난 점심하러 가겠습니다~,27,5,NaN,NaN,NaN,NaN,0.0
1,2022년 10월 24일 오전 10:48,2022년 10월 24일 AM 10:48,조윤실,청계산 청룡공영주차장\n서울 서초구 신원동 224-3,28,6,NaN,NaN,NaN,NaN,0.0
2,2022년 10월 24일 오전 10:49,2022년 10월 24일 AM 10:49,조윤실,수경샘~ 작은 주차장이지만 여기에 주차하시면됩니다.,30,5,NaN,NaN,NaN,NaN,0.0
3,2022년 10월 24일 오전 10:50,2022년 10월 24일 AM 10:50,조윤실,저는 12시 10분 땡퇴하고 가겠습니다. 25~30분 정도 걸릴거에요.,39,8,NaN,NaN,NaN,NaN,0.0
4,2022년 10월 24일 오전 11:09,2022년 10월 24일 AM 11:09,TMD-최수경-샘,넹!! 곧 뵈어요,9,3,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...
244,2023년 3월 22일 오전 11:34,2023년 3월 22일 AM 11:34,TMD-최수경-샘,그 친구는 만들고 싶은 서비스가 많다니까 ㅎㅎ 저는 걍 구경하고 시키는 일하려고요,45,12,NaN,NaN,NaN,NaN,0.0
245,2023년 3월 22일 오전 11:35,2023년 3월 22일 AM 11:35,TMD-최수경-샘,점심 맛나게들 챙겨드세욤,13,3,NaN,NaN,NaN,NaN,0.0
246,2023년 3월 22일 오전 11:36,2023년 3월 22일 AM 11:36,송혜경,이모티콘,4,1,NaN,NaN,NaN,NaN,0.0
247,2023년 3월 22일 오전 11:39,2023년 3월 22일 AM 11:39,조윤실,난 지금부터 일 시작!!!,14,4,NaN,NaN,NaN,NaN,0.0


In [ ]:
# 메시지 건수 조정 (동영상/사진 등을 메시지로 인식한 경우 메시지 건수 0)
df.loc[mask, 'msg_len'] = 0
df.loc[mask, 'msg_word_count'] = 0
df[df['audio_visual']==1]

### 통화 정보 추출하기
- 페이스톡 해요 (Face Call) : 통화 걸기
- 페이스톡 취소 (Cancelled Face Call): 통화 취소
- 페이스톡 부재중 (Missed Video Call): 상대방이 걸어왔지만 내가 응답 없었을 때
- 페이스톡 응답없음 (No Answer on Video Call): 내가 걸었지만 상대방이 응답 없었을 때
- 페이스톡 8:02 (Video Call 8:02): 10분 이내 통화
- 페이스톡 26:25 (Video Call 26:25): 10분 이상 통화
- 페이스톡 1:30:19 (Video Call 1:30:19): 1시간 이상 통화

In [ ]:
df[df['text'].str.contains('보이스톡|페이스톡')]

In [19]:
def get_call_length(call_df):

    call_mask = call_df.text.str.contains('해요|취소|부재중|응답없음')  # 0
    call_df = call_df.text.str.extract('([0-9]{1,2}:)*([0-9]{1,2}):([0-9]{2})')  # 1
    call_df[0] = call_df[0].str.replace(':', '')  # 2
    call_df[0] = call_df[0].fillna(0)  # 3
    call_df = call_df.astype(float)  # 4
    call_df['call_len'] = call_df[0] * 3660 + call_df[1] * 60 + call_df[2]  # 5
    call_df.loc[call_mask, 'call_len'] = 0  # 6

    return call_df[['call_len']]  # 7

call_col_dict = {
    '페이스톡|Video Call': 'facetalk',
    '보이스톡|Voice Call': 'voicetalk',
}


for call in call_col_dict:
    mask = df['text'].str.contains(call)
    call_df = get_call_length(df[mask])
    call_df = call_df.rename(columns={'call_len': call_col_dict[call]})
    call_df.head()

    df = pd.concat([df, call_df], axis=1)

In [15]:
mask = (df['voicetalk'] >= 0) | (df['facetalk'] >= 0)
df.loc[mask, 'msg_len'] = 0
df.loc[mask, 'msg_word_count'] = 0

KeyError: ignored

In [ ]:
df[df['text'].str.contains('보이스톡|페이스톡')]

In [17]:
df['voicetalk'].dtype

AttributeError: ignored

### 비언어 표현 추출하기(이모티콘 등)

| 비언어 표현 | 텍스트 데이터 예시 | 정규표현식 |
|--|--|--|
|웃음, 울음 (ㅋㅋ, ㅎㅎ)  |ㅋㅋ, ㅋㅋㅋㅋ, ㅎㅎㅎㅎㅎ  |[ㅋㅎㅠㅜ]+  |
|느낌표 (!) , 물음표 (?)  |!, !!!!!!!, !?!?, ?????  |[!?~]+  |
| 쉼표(,), 점(.) | ……. ,,,,,, | [,.]{2,} |
| 이모티콘 타입 1 (입 중심)  | :) :D | 	[;:]{1}-?[)(DPpboOX] |
 |이모티콘 타입 1 (눈 중심)  | ^^, ㅜㅜ | ([>ㅜㅠㅡ@\^-+][ㅁㅇ0oO._-] *[\^ㅜㅠㅡ@<-+<];) |
| 이모티콘 타입 2 | 😃😆😍❤ | 정규표현식 대신 emoji 라이브러리 사용 |
| 이모티콘 타입 3 | (반함) (굿) (찡긋) (이모티콘 |  ((.*?)+) |

In [11]:
df

,date_time,datetime,user_name,text,msg_len,msg_word_count,facetalk,voicetalk,facetalk,voicetalk
0,2022년 10월 24일 오전 10:48,2022년 10월 24일 AM 10:48,조윤실,혜경샘~~오늘 수경샘과 맛난 점심하러 가겠습니다~,27,5,NaN,NaN,NaN,NaN
1,2022년 10월 24일 오전 10:48,2022년 10월 24일 AM 10:48,조윤실,청계산 청룡공영주차장\n서울 서초구 신원동 224-3,28,6,NaN,NaN,NaN,NaN
2,2022년 10월 24일 오전 10:49,2022년 10월 24일 AM 10:49,조윤실,수경샘~ 작은 주차장이지만 여기에 주차하시면됩니다.,30,5,NaN,NaN,NaN,NaN
3,2022년 10월 24일 오전 10:50,2022년 10월 24일 AM 10:50,조윤실,저는 12시 10분 땡퇴하고 가겠습니다. 25~30분 정도 걸릴거에요.,39,8,NaN,NaN,NaN,NaN
4,2022년 10월 24일 오전 11:09,2022년 10월 24일 AM 11:09,TMD-최수경-샘,넹!! 곧 뵈어요,9,3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
244,2023년 3월 22일 오전 11:34,2023년 3월 22일 AM 11:34,TMD-최수경-샘,그 친구는 만들고 싶은 서비스가 많다니까 ㅎㅎ 저는 걍 구경하고 시키는 일하려고요,45,12,NaN,NaN,NaN,NaN
245,2023년 3월 22일 오전 11:35,2023년 3월 22일 AM 11:35,TMD-최수경-샘,점심 맛나게들 챙겨드세욤,13,3,NaN,NaN,NaN,NaN
246,2023년 3월 22일 오전 11:36,2023년 3월 22일 AM 11:36,송혜경,이모티콘,4,1,NaN,NaN,NaN,NaN
247,2023년 3월 22일 오전 11:39,2023년 3월 22일 AM 11:39,조윤실,난 지금부터 일 시작!!!,14,4,NaN,NaN,NaN,NaN


In [21]:
# 모두 포함하는 정규 표현식
# (\(.+?\)) | ([ㅋㅎㅠㅜ!?~]+) | [,.]{2,} | ([;:]{1}[\^\'-]?[)(DPpboOX]) | ([>ㅜㅠㅡ@+\^][ㅁㅇ0oO\._\-]*[\^ㅜㅠㅡ@+<];*)

In [20]:
# 이모지 라이브러리 설치
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 4.0 MB/s eta 0:00:00


In [22]:
import emoji

def extract_emojis(text):
    emoji_list = list()
    for c in text:
#         if c in emoji.UNICODE_EMOJI['en']:
        if c in emoji.EMOJI_DATA:
            emoji_list.append(c)

    return emoji_list

mimetic= "[ㅋㅎㅠㅜ!?~]+"
punctuations = "[,.]{2,}"
emo_type1_facial1 = "[;:]{1}[\^\'-]?[)(DPpboOX]"
emo_type1_facial2 = "[>ㅜㅠㅡ@\^][ㅁㅇ0oO\._\-]*[\^ㅜㅠㅡ@<];*"
emo_type3 = "\(.+?\)"

# 1개ㅔ 이상의 비언어 표현을 가지고 있는 데이터 출력하기
nonverbal_list = [mimetic, punctuations, emo_type1_facial1, emo_type1_facial2, emo_type3]

df['nonverbal'] = df['text'].str.findall('|'.join(nonverbal_list)) + df['text'].map(extract_emojis)
df['nonverbal_count'] = df['nonverbal'].apply(len)
df.loc[df['nonverbal_count'] > 0]


,date_time,datetime,user_name,text,msg_len,msg_word_count,facetalk,voicetalk,facetalk,voicetalk,audio_visual,facetalk,voicetalk,facetalk,voicetalk,nonverbal,nonverbal_count
0,2022년 10월 24일 오전 10:48,2022년 10월 24일 AM 10:48,조윤실,혜경샘~~오늘 수경샘과 맛난 점심하러 가겠습니다~,27,5,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"[~~, ~]",2
2,2022년 10월 24일 오전 10:49,2022년 10월 24일 AM 10:49,조윤실,수경샘~ 작은 주차장이지만 여기에 주차하시면됩니다.,30,5,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[~],1
3,2022년 10월 24일 오전 10:50,2022년 10월 24일 AM 10:50,조윤실,저는 12시 10분 땡퇴하고 가겠습니다. 25~30분 정도 걸릴거에요.,39,8,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[~],1
4,2022년 10월 24일 오전 11:09,2022년 10월 24일 AM 11:09,TMD-최수경-샘,넹!! 곧 뵈어요,9,3,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[!!],1
7,2022년 10월 24일 오전 11:25,2022년 10월 24일 AM 11:25,TMD-최수경-샘,네넹! ^^,6,2,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"[!, ^^]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,2023년 3월 22일 오전 11:32,2023년 3월 22일 AM 11:32,TMD-최수경-샘,ㅋㅋㅋ 제안서에 넣고 고객사에 샘플로 만들페이지 만들게 할거라며 어제 종일 일도 못...,76,17,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"[ㅋㅋㅋ, ㅋㅋㅋㅋㅋㅋ]",2
242,2023년 3월 22일 오전 11:33,2023년 3월 22일 AM 11:33,송혜경,난 ppt로 하는데. ㅋ,13,4,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[ㅋ],1
243,2023년 3월 22일 오전 11:34,2023년 3월 22일 AM 11:34,TMD-최수경-샘,저도 PPT가 빠를 듯.. 그건 대표의 꿈이고,25,7,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[..],1
244,2023년 3월 22일 오전 11:34,2023년 3월 22일 AM 11:34,TMD-최수경-샘,그 친구는 만들고 싶은 서비스가 많다니까 ㅎㅎ 저는 걍 구경하고 시키는 일하려고요,45,12,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[ㅎㅎ],1


#### 이모티콘 사용

In [31]:
# 이모티콘 사용한 레코드 변경
df.loc[df['text'] == '이모티콘', 'nonverbal'] = '[emoticon]'
df.loc[df['text'] == '이모티콘', 'nonverbal_count'] = 1
df.loc[df['text'] == '이모티콘', 'msg_len'] = 0
df.loc[df['text'] == '이모티콘', 'msg_word_count'] = 0
df.loc[df['text'] == '이모티콘']

,date_time,datetime,user_name,text,msg_len,msg_word_count,facetalk,voicetalk,facetalk,voicetalk,audio_visual,facetalk,voicetalk,facetalk,voicetalk,nonverbal,nonverbal_count
44,2022년 11월 14일 오후 3:25,2022년 11월 14일 PM 3:25,최수경,이모티콘,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[emoticon],1
58,2022년 11월 28일 오후 1:00,2022년 11월 28일 PM 1:00,송혜경,이모티콘,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[emoticon],1
71,2022년 11월 28일 오후 1:05,2022년 11월 28일 PM 1:05,최수경,이모티콘,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[emoticon],1
90,2022년 12월 12일 오후 12:17,2022년 12월 12일 PM 12:17,송혜경,이모티콘,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[emoticon],1
128,2023년 1월 18일 오전 9:13,2023년 1월 18일 AM 9:13,최수경,이모티콘,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[emoticon],1
153,2023년 1월 31일 오후 7:26,2023년 1월 31일 PM 7:26,조윤실,이모티콘,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[emoticon],1
198,2023년 3월 3일 오후 2:35,2023년 3월 3일 PM 2:35,조윤실,이모티콘,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[emoticon],1
199,2023년 3월 3일 오후 2:35,2023년 3월 3일 PM 2:35,송혜경,이모티콘,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[emoticon],1
228,2023년 3월 22일 오전 11:24,2023년 3월 22일 AM 11:24,송혜경,이모티콘,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[emoticon],1
246,2023년 3월 22일 오전 11:36,2023년 3월 22일 AM 11:36,송혜경,이모티콘,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[emoticon],1


In [29]:
df.loc[df['text'] == '이모티콘'].count()

date_time          11
datetime           11
user_name          11
text               11
msg_len            11
msg_word_count     11
facetalk            0
voicetalk           0
facetalk            0
voicetalk           0
audio_visual       11
facetalk            0
voicetalk           0
facetalk            0
voicetalk           0
nonverbal          11
nonverbal_count    11
dtype: int64

### URL 추출하기

In [32]:
# URL 추출 라이브러리 설치하기
!pip install urlextract

In [33]:
# URL 추출 예제
from urlextract import URLExtract

text = """Google Colab: https://colab.research.google.com.
       Introduction for google colab: https://colab.research.google.com/notebooks/intro.ipynb"""

extractor = URLExtract()
urls = extractor.find_urls(text)

for url in urls:
    print(url)

https://colab.research.google.com
https://colab.research.google.com/notebooks/intro.ipynb


In [34]:
from urlextract import URLExtract


extractor = URLExtract()

df['url'] = df['text'].apply(extractor.find_urls)


df['url_count'] = df['url'].apply(len)
url_df = df.loc[df['url_count'] > 0]
url_df[['date_time','user_name','url','url_count']]


,date_time,user_name,url,url_count
26,2022년 11월 8일 오후 8:49,조윤실,[https://youtu.be/UVJndTVGsPU],1
47,2022년 11월 17일 오후 5:15,최수경,[https://russel.megastudy.net/intro/2022/csat/...,1
105,2022년 12월 12일 오후 8:51,최수경,[http://www.edupress.kr/news/articleView.html?...,1


### 특정 조건의 컬럼 값 변경

In [35]:
df.loc[df['user_name'] == 'TMD-최수경-샘', 'user_name'] = '최수경'

### 데이터 저장하기

In [26]:
fname = './sample_data/KakaoTalkChats_데이터분석_결과.csv'
df.to_csv(fname)

In [ ]:
df = pd.read_csv(fname, encoding='utf-8')
df

----------------------------------------

## 4. (기술통계)데이터 시각화
- https://wikidocs.net/162798

### 일자별 선/막대 그래프 (plotly)

In [38]:
import plotly.express as px

# 선그래프
fig = px.line(df, x='datetime', title='카카오톡 일자별 메시지')
fig

In [40]:
# 막대 그래프
fig = px.bar(df, x='date_time', title='카카오톡 일자별 메시지')
fig

In [41]:
# 점그래프
fig = px.scatter(df, x='date_time', title='카카오톡 일자별 메시지')
fig

### 사용자별 메시지 건수

In [42]:
# 유니크한 사용자
df['user_name'].unique()

array(['조윤실', '최수경', '송혜경'], dtype=object)

In [43]:
# 사용자별 메시지 건수(default  파레토(Pareto) 분포: 내림차순 정렬)
df['user_name'].value_counts()

최수경    112
송혜경     75
조윤실     62
Name: user_name, dtype: int64

In [44]:
# 사용자별 메시지 건수
fig = px.bar(df['user_name'].value_counts())
fig.show()

### 요일별 / 시간별 메시지 건수

In [ ]:
# 요일별 메시지 건수
df['weekday'].value_counts()

In [ ]:
# 시간별 메시지 건수
df['hour'].value_counts()

In [ ]:
# 시간별 메시지 건수(인덱스 순서대로 정렬)
df['hour'].value_counts().sort_index()

In [ ]:
# 시간별 메시지 건수
fig = px.bar(df['hour'].value_counts().sort_index())
fig.show()

### 일자별 히스토그램 : 슬라이더 그래프

In [ ]:
# 일자별 히스토그램
fig = px.histogram(df, x='date')
fig.show()

In [ ]:
# 슬라이더 그래프 생성: 확대해서 보여준다.
fig = px.histogram(df, x='date')

fig.update_xaxes(
    rangeslider_visible=True,
)
fig.show()

In [ ]:
# 사용자별 히스토그램
fig = px.histogram(df, x='date', color='user_name')
fig.show()

------------

THE END